In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import random
import numpy as np
import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn

import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.optim import Adam

In [3]:
pd.options.mode.chained_assignment = None  # default='warn'
pd.options.display.max_rows = 999
pd.options.display.max_columns = 100

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


## Load data

In [5]:
import joblib

df_samples = joblib.load('data/amazon_data.pkl')

In [6]:
len( df_samples['reviewerID'].unique() )

192403

In [7]:
len( df_samples['asin'].unique() )

63001

In [8]:
len( df_samples['brand'].unique() )

3526

In [9]:
len( df_samples['categories'].unique() )

801

## Create pytorch dataset

In [27]:
## Hash buckets
# feat_configs = [
#     {"name": "reviewerID", "dtype": "category", "emb_dim": 12, "min_freq": 3, "hash_buckets": 1000000},
#     {"name": "asin", "dtype": "category", "emb_dim": 12, "min_freq": 3, "hash_buckets": 1000000},
    
#     {"name": "price", "dtype": "numerical", "norm": "std"},
#     {"name": "brand", "dtype": "category", "min_freq": 3, "emb_dim": 12},
#     {"name": "categories", "dtype": "category", "min_freq": 3, "emb_dim": 12},

#     {"name": "his_asin_seq", "dtype": "category", "islist": True, "min_freq": 3, "emb_dim": 12, "hash_buckets": 1000000},
# ]

## Dynamic Embedding
feat_configs = [
    {"name": "reviewerID", "dtype": "category", "emb_dim": 12, "min_freq": 3},
    {"name": "asin", "dtype": "category", "emb_dim": 12, "min_freq": 3},
    
    {"name": "price", "dtype": "numerical", "norm": "std"},
    {"name": "brand", "dtype": "category", "min_freq": 3, "emb_dim": 12},
    {"name": "categories", "dtype": "category", "min_freq": 3, "emb_dim": 12},

    {"name": "his_asin_seq", "dtype": "category", "islist": True, "min_freq": 3, "emb_dim": 12},
]

target_cols = ['label', ]

In [28]:
from core.sample import traintest_split

df_train, df_test = traintest_split(df_samples, test_size=0.2, shuffle=True, group_id='reviewerID')
print(len(df_train), len(df_test))

1352538 336650


In [29]:
# from core.dataset import FeatureTransformer

# transformer = FeatureTransformer(feat_configs)

# df_train = transformer.transform(df_train, is_train=True, n_jobs=4)
# df_test = transformer.transform(df_test, is_train=False, n_jobs=4)

In [30]:
from core.dataset import DataFrameDataset

train_dataset = DataFrameDataset(df_train, feat_configs, target_cols, is_raw=True, is_train=True, n_jobs=1, verbose=True)
test_dataset = DataFrameDataset(df_test, feat_configs, target_cols, is_raw=True, is_train=False, n_jobs=1)

==> Feature transforming (is_train=True), note that feat_configs will be updated when is_train=True...
Processing feature reviewerID...
Converting category reviewerID to indices...
Feature reviewerID vocab size: None -> 153923
Processing feature asin...
Converting category asin to indices...
Feature asin vocab size: None -> 62384
Processing feature price...
Feature price mean: 74.35263034595687, std: 123.73053965875545, min: 0.01, max: 999.99
Processing feature brand...
Converting category brand to indices...
Feature brand vocab size: None -> 3419
Processing feature categories...
Converting category categories to indices...
Feature categories vocab size: None -> 800
Processing feature his_asin_seq...
Converting category his_asin_seq to indices...
Feature his_asin_seq vocab size: None -> 61925
==> Feature transforming (is_train=True) done...
==> Dense features: ['price']
==> Sparse features: ['reviewerID', 'asin', 'brand', 'categories']
==> Sequence dense features: []
==> Sequence spars

In [32]:
# max([v['idx'] for k,v in feat_configs[3]['vocab'].items()])
# feat_configs[3]['num_embeddings']

3419

In [33]:
df_train.head()

,reviewerID,asin,unixReviewTime,overall,title,price,brand,categories,label,his_asin_seq
314370,69777,1870,1375660800,4.0,B+W 58mm Kaesemann Circular Polarizer with Mul...,0.118381,153,197,1,"[-100, -100, -100, -100, -100, -100, -100, -10..."
538816,47184,140,1342137600,5.0,OtterBox Defender Series Case with Screen Prot...,-0.075831,77,1,1,"[-100, -100, -100, -100, -100, -100, -100, -10..."
194675,111551,2570,1397088000,5.0,Olympus VN-702PC Voice Recorder,-0.124243,54,140,1,"[-100, -100, -100, -100, -100, -100, -100, -10..."
543055,10936,6521,1357516800,4.0,Manfrotto 701HDV Pro Fluid Video Mini Head,1.823619,1,153,1,"[-100, -100, -100, -100, -100, -100, -100, -10..."
355447,99991,3273,1365724800,4.0,Bear Motion Luxury Buffalo Hide Vintage Leathe...,-0.196901,546,1,1,"[-100, -100, -100, -100, -100, -100, -100, -10..."


In [34]:
train_dataloader = DataLoader(train_dataset, batch_size=512, num_workers=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=512, num_workers=8, shuffle=False)

In [35]:
print( len(train_dataloader) )
for features, labels in DataLoader(train_dataset,batch_size=1,shuffle=True):
    print(features)
    print(labels)
    break

2642
{'dense_features': tensor([[-0.5687]]), 'reviewerID': tensor([[25]], dtype=torch.int32), 'asin': tensor([[34173]], dtype=torch.int32), 'brand': tensor([[1]], dtype=torch.int32), 'categories': tensor([[70]], dtype=torch.int32), 'his_asin_seq': tensor([[ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -10

## Train Model

In [36]:
from model import DNN

dnn_hidden_units = [128,64,32]
model = DNN(feat_configs, hidden_units=dnn_hidden_units)
model = model.to(device)
print(model)

==> Model Input: dense_size=1, sparse_size=60
DNN(
  (embeddings): ModuleDict(
    (reviewerID): DynamicEmbedding(153923, 12)
    (asin): DynamicEmbedding(62384, 12)
    (brand): DynamicEmbedding(3420, 12)
    (categories): DynamicEmbedding(800, 12)
    (his_asin_seq): DynamicEmbedding(61926, 12)
  )
  (fc_layers): Sequential(
    (0): Linear(in_features=61, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=64, out_features=32, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.2, inplace=False)
  )
  (logits): Linear(in_features=32, out_features=1, bias=True)
)


In [37]:
optimizer = Adam(model.parameters(),  lr = 0.001, weight_decay = 1e-9)
lr_scd = lr_scheduler.StepLR(optimizer, step_size=len(train_dataloader), gamma=0.8)

In [38]:
from core.trainer import Trainer

trainer = Trainer(
    model, 
    optimizer=optimizer,
    lr_scheduler=lr_scd,
    max_epochs=5,
    early_stopping_rounds=3,
    save_ckpt_path='./ckpt/'
)

model = trainer.fit(train_dataloader, eval_dataloader = test_dataloader, ret_model = 'final') #, init_ckpt_path='./ckpt/')

INFO:Trainer:[Validation] Epoch: 0/5, Validation Loss: {'loss': 0.755312853399381}
INFO:Trainer:Learning rate: 0.001
INFO:Trainer:[Training] Epoch: 1/5 iter 0/2642, Training Loss: {'loss': 0.7546861171722412}
INFO:Trainer:[Training] Epoch: 1/5 iter 100/2642, Training Loss: {'loss': 0.4085532212257385}
INFO:Trainer:[Training] Epoch: 1/5 iter 200/2642, Training Loss: {'loss': 0.38834586724638936}
INFO:Trainer:[Training] Epoch: 1/5 iter 300/2642, Training Loss: {'loss': 0.37921916236480074}
INFO:Trainer:[Training] Epoch: 1/5 iter 400/2642, Training Loss: {'loss': 0.37412176474928854}
INFO:Trainer:[Training] Epoch: 1/5 iter 500/2642, Training Loss: {'loss': 0.3707314270734787}
INFO:Trainer:[Training] Epoch: 1/5 iter 600/2642, Training Loss: {'loss': 0.3680782702565193}
INFO:Trainer:[Training] Epoch: 1/5 iter 700/2642, Training Loss: {'loss': 0.36574090263673237}
INFO:Trainer:[Training] Epoch: 1/5 iter 800/2642, Training Loss: {'loss': 0.36481436014175417}
INFO:Trainer:[Training] Epoch: 1/5

In [39]:
ckpt = trainer.load_ckpt('./ckpt')
model.load_state_dict(ckpt['model'].state_dict())

INFO:Trainer:Loaded model state_dict from checkpoint.
INFO:Trainer:Loaded model.training from checkpoint.
INFO:Trainer:Loaded model.feat_configs from checkpoint.
INFO:Trainer:Loaded optimizer = Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differ... from checkpoint.
INFO:Trainer:Loaded lr_scheduler = <torch.optim.lr_scheduler.StepLR object at 0x2a9e546a0> from checkpoint.
INFO:Trainer:Loaded logger = <Logger Trainer (INFO)> from checkpoint.
INFO:Trainer:Loaded ckpt_file_prefix = checkpoint from checkpoint.
INFO:Trainer:Loaded num_epoch = 4 from checkpoint.
INFO:Trainer:Loaded global_steps = 10568 from checkpoint.
INFO:Trainer:Loaded save_ckpt_path = ./ckpt/ from checkpoint.
INFO:Trainer:Loaded metadata_fn = ./ckpt//metadata.json from checkpoint.
INFO:Trainer:Loaded max_epochs = 5 from checkpoint.
INFO:Trainer:Loaded early_stopping_rounds = 3 from checkpoint.
INFO:Trainer:Checkpoint loaded from ./ckpt/checkpoint.010568.ckpt.


<All keys matched successfully>

In [40]:
test_preds = []
test_labels = []
model.eval()

for features, labels in test_dataloader:
    outputs = model(features)
    test_preds.append(outputs[:,0])
    test_labels.append(labels[:,0])
test_preds = torch.concat(test_preds, dim=0).detach().cpu().numpy()
test_labels = torch.concat(test_labels, dim=0).detach().cpu().numpy()

In [41]:
print(test_preds.shape, test_labels.shape)

(336650,) (336650,)


In [42]:
from sklearn.metrics import roc_auc_score

auc_score = roc_auc_score(test_labels, test_preds)
print("AUC Score:", auc_score)

AUC Score: 0.6673137778411582
